In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


def plot_results_regression(df, metric="mae_test"):

    df["gaussian"] = pd.to_numeric(df["gaussian"], errors="coerce")

    cov_types = df["covariance"].unique()
    for cov in cov_types:
        df_cov = df[df["covariance"] == cov]

        plt.figure(figsize=(14, 6))
        sns.barplot(
            data=df_cov,
            x="gaussian",
            y=metric,
            hue="model_name",
            palette="tab10"
        )

        plt.title(f"{metric} por número de gaussianas - Covariance: {cov}")
        plt.xlabel("Número de gaussianas")
        plt.ylabel(metric)
        plt.legend(title="Modelo")
        plt.grid(axis="y", linestyle="--", alpha=0.5)

        plt.tight_layout()
        plt.show()
def plot_results_classification(df, metric="acc_test", n_classes = 2):
    # Asegurar que gaussians sea numérico
    df["gaussian"] = pd.to_numeric(df["gaussian"], errors="coerce")

    cov_types = df["covariance"].unique()

    for cov in cov_types:
        df_cov = df[df["covariance"] == cov]

        plt.figure(figsize=(14, 6))
        sns.barplot(
            data=df_cov,
            x="gaussian",
            y=metric,
            hue="model_name",
            palette="tab10"
        )

        plt.title(f"{metric} according to the number of Gaussians - Covariance: {cov} - Classes {n_classes}")
        plt.xlabel("Number of Gaussians")
        plt.ylabel(metric)
        plt.legend(title="Model")
        plt.grid(axis="y", linestyle="--", alpha=0.5)

        plt.show()


In [ ]:
PATH = "/home/administrador/Documents/Santiago/results/results/run_11_09_2130/0_0/reg_results_wo.csv"
# === 1. Cargar CSV ===
df = pd.read_csv(PATH)
plot_results_regression(df, "mae_test")

In [ ]:
PATH = "/home/administrador/Documents/Santiago/results/results/run_11_09_2130/0_0/reg_results_w.csv"
# === 1. Cargar CSV ===
df = pd.read_csv(PATH)
plot_results_regression(df, "mae_test")

In [ ]:
PATH = "/home/administrador/Documents/Santiago/results/results_classification/0_0/run_11_09_2129/class_results_w_2classes.csv"
# === 1. Cargar CSV ===
df = pd.read_csv(PATH)
plot_results_classification(df, "acc_test")

## Tabla de Resultados

In [2]:
import pandas as pd
import os

In [86]:
def reg_tabla_mejores_resultados(root, subfolder, n_best=1):

    folder = os.path.join(root, subfolder)
    results = pd.DataFrame()
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            filepath = os.path.join(folder, filename)
            df = pd.read_csv(filepath)
            best_results = df.nsmallest(n_best, 'mae_test')
            #poner en primera columna el nombre del archivo
            best_results.insert(0, 'file', filename)
            results = pd.concat([results, best_results[['file', 'model_name', 'gaussian', 'covariance', 'mae_test']]], ignore_index=True)
    display(results)

def class_tabla_mejores_resultados(root, subfolder, n_best=1):

    folder = os.path.join(root, subfolder)
    results = pd.DataFrame()
    files = ['class_results_w_2classes.csv','class_results_wo_2classes.csv']
    for filename in files:
        filepath = os.path.join(folder, filename)
        df = pd.read_csv(filepath)
        best_results = df.nlargest(n_best, 'acc_test')
        #poner en primera columna el nombre del archivo
        best_results.insert(0, 'file', filename)
        results = pd.concat([results, best_results[['file', 'model_name', 'gaussian', 'covariance', 'acc_test']]], ignore_index=True)
    display(results)


def reg_tabla_models_desv(root, subfolder, metric="mae_test"):
    """
    Agrupar por modelos, poner el metrica promedio y calcular la desv std entre sus resultados
    """

    folder = os.path.join(root, subfolder)
    results = pd.DataFrame()
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            filepath = os.path.join(folder, filename)
            df = pd.read_csv(filepath)
            # agrupar por modelo y calcular promedio y desviacion estandar
            results = pd.concat([results, df[['model_name', metric]]], ignore_index=True)
            # Agrupar por modelo y calcular promedio y desviacion estandar para este archivo
            
            final_results = results.groupby('model_name').agg(
                mae_test_mean=(metric, 'mean'),
                mae_test_std=(metric, 'std')
            ).reset_index()
            
            # poner en primera columna el nombre del archivo
            final_results.insert(0, 'file', filename)
            # 
            display(final_results)
def reg_tabla_veces_ganador(root, subfolder, metric="mae_test"):
    """
    Para cada combinación (gaussian, covariance) se considera ganador el modelo con menor mae_test
    Para cada model_name se calcula:
        Número de veces que ganó
    """
    folder = os.path.join(root, subfolder)
    results = pd.DataFrame()
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            filepath = os.path.join(folder, filename)
            df = pd.read_csv(filepath)
            # Agrupar por gaussian y covariance, obtener el modelo con menor mae_test
            best_results = df.loc[df.groupby(['gaussian', 'covariance'])[metric].idxmin()]
            # Contar número de veces que cada modelo ganó
            win_counts = best_results['model_name'].value_counts().reset_index()
            win_counts.columns = ['model_name', 'win_count']
            # poner en primera columna el nombre del archivo
            win_counts.insert(0, 'file', filename)
            results = pd.concat([results, win_counts], ignore_index=True)
    display(results)

def class_tabla_models_desv(root, subfolder, metric="acc_test"):
    """
    Agrupar por modelos, poner el metrica promedio y calcular la desv std entre sus resultados
    """

    folder = os.path.join(root, subfolder)
    results = pd.DataFrame()
    files = ['class_results_w_2classes.csv','class_results_wo_2classes.csv']
    for filename in files:
        filepath = os.path.join(folder, filename)
        df = pd.read_csv(filepath)
        # agrupar por modelo y calcular promedio y desviacion estandar
        results = pd.concat([results, df[['model_name', metric]]], ignore_index=True)
        # Agrupar por modelo y calcular promedio y desviacion estandar para este archivo
        
        final_results = results.groupby('model_name').agg(
            acc_test_mean=(metric, 'mean'),
            acc_test_std=(metric, 'std')
        ).reset_index()
        
        # poner en primera columna el nombre del archivo
        final_results.insert(0, 'file', filename)
        # 
        display(final_results)

def class_tabla_veces_ganador(root, subfolder, metric="acc_test"):
    """
    Para cada combinación (gaussian, covariance) se considera ganador el modelo con mayor acc_test
    Para cada model_name se calcula:
        Número de veces que ganó
    """
    folder = os.path.join(root, subfolder)
    results = pd.DataFrame()
    files = ['class_results_w_2classes.csv','class_results_wo_2classes.csv']
    for filename in files:
        filepath = os.path.join(folder, filename)
        df = pd.read_csv(filepath)
        # Agrupar por gaussian y covariance, obtener el modelo con mayor acc_test
        best_results = df.loc[df.groupby(['gaussian', 'covariance'])[metric].idxmax()]
        # Contar número de veces que cada modelo ganó
        win_counts = best_results['model_name'].value_counts().reset_index()
        win_counts.columns = ['model_name', 'win_count']
        # poner en primera columna el nombre del archivo
        win_counts.insert(0, 'file', filename)
        results = pd.concat([results, win_counts], ignore_index=True)
    display(results)

In [30]:
PATH_REG = R'D:\Semillero SOFA\gmm_32_definitivo\results_regression_24_11_newparams\results_regression\run_11_22_1132'

### Mejores Resultados Regression

In [ ]:
reg_tabla_mejores_resultados(PATH_REG,'0_0')

,file,model_name,gaussian,covariance,mae_test
0,reg_results_w.csv,RandomForest,16,diag,1.882021
1,reg_results_wo.csv,SVM,64,spherical,2.443723


In [ ]:
reg_tabla_mejores_resultados(PATH_REG,'270_0')

,file,model_name,gaussian,covariance,mae_test
0,reg_results_w.csv,RandomForest,16,spherical,1.229187
1,reg_results_wo.csv,SVM,64,spherical,2.403365


In [ ]:
reg_tabla_mejores_resultados(PATH_REG,'270_9')

,file,model_name,gaussian,covariance,mae_test
0,reg_results_w.csv,RandomForest,16,spherical,1.435204
1,reg_results_wo.csv,SVM,64,spherical,2.607209


### Resultados con Desv std

In [ ]:
reg_tabla_models_desv(PATH_REG,'0_0')

,file,model_name,mae_test_mean,mae_test_std
0,reg_results_w.csv,DecisionTree,2.314218,0.130960
1,reg_results_w.csv,RandomForest,2.043437,0.092586
2,reg_results_w.csv,SVM,2.121704,0.017643


,file,model_name,mae_test_mean,mae_test_std
0,reg_results_wo.csv,DecisionTree,2.614289,0.320332
1,reg_results_wo.csv,RandomForest,2.413109,0.382081
2,reg_results_wo.csv,SVM,2.299366,0.182384


In [ ]:
reg_tabla_models_desv(PATH_REG,'270_0')

,file,model_name,mae_test_mean,mae_test_std
0,reg_results_w.csv,DecisionTree,2.036428,0.226932
1,reg_results_w.csv,RandomForest,1.576676,0.186982
2,reg_results_w.csv,SVM,1.967532,0.016755


,file,model_name,mae_test_mean,mae_test_std
0,reg_results_wo.csv,DecisionTree,2.442390,0.444564
1,reg_results_wo.csv,RandomForest,2.112683,0.561117
2,reg_results_wo.csv,SVM,2.201907,0.239481


In [ ]:
reg_tabla_models_desv(PATH_REG,'270_9')

,file,model_name,mae_test_mean,mae_test_std
0,reg_results_w.csv,DecisionTree,2.235005,0.242951
1,reg_results_w.csv,RandomForest,1.829056,0.239432
2,reg_results_w.csv,SVM,2.166322,0.012323


,file,model_name,mae_test_mean,mae_test_std
0,reg_results_wo.csv,DecisionTree,2.717333,0.520484
1,reg_results_wo.csv,RandomForest,2.439640,0.643646
2,reg_results_wo.csv,SVM,2.400241,0.238797


### Cuenta de Veces ganadas

In [87]:
reg_tabla_veces_ganador(PATH_REG,'0_0')

,file,model_name,win_count
0,reg_results_w.csv,RandomForest,11
1,reg_results_w.csv,SVM,3
2,reg_results_wo.csv,SVM,14


In [88]:
reg_tabla_veces_ganador(PATH_REG,'270_0')

,file,model_name,win_count
0,reg_results_w.csv,RandomForest,14
1,reg_results_wo.csv,SVM,14


In [89]:
reg_tabla_veces_ganador(PATH_REG,'270_9')

,file,model_name,win_count
0,reg_results_w.csv,RandomForest,13
1,reg_results_w.csv,SVM,1
2,reg_results_wo.csv,SVM,14


## Tabla Clasificación

In [39]:
PATH_CLASS = R'D:\Semillero SOFA\gmm_32_definitivo\results_classification2412\results_classification'
#run_12_10_2117
#run_11_22_1156

### Top Result

In [71]:
class_tabla_mejores_resultados(PATH_CLASS,'0_0/run_11_22_1156')

,file,model_name,gaussian,covariance,acc_test
0,class_results_w_2classes.csv,DecisionTree,16,spherical,0.811591
1,class_results_wo_2classes.csv,RandomForest,56,diag,0.726419


In [72]:
class_tabla_mejores_resultados(PATH_CLASS,'270_0/run_11_22_1156')

,file,model_name,gaussian,covariance,acc_test
0,class_results_w_2classes.csv,DecisionTree,16,spherical,0.827656
1,class_results_wo_2classes.csv,RandomForest,48,spherical,0.726637


In [73]:
class_tabla_mejores_resultados(PATH_CLASS,'270_9/run_11_22_1156')

,file,model_name,gaussian,covariance,acc_test
0,class_results_w_2classes.csv,RandomForest,16,diag,0.817524
1,class_results_wo_2classes.csv,RandomForest,64,spherical,0.709243


### Tabla Desv Std

In [76]:
class_tabla_models_desv(PATH_CLASS,'0_0/run_11_22_1156', 'acc_test')

,file,model_name,acc_test_mean,acc_test_std
0,class_results_w_2classes.csv,DecisionTree,0.754895,0.027122
1,class_results_w_2classes.csv,RandomForest,0.742347,0.009634
2,class_results_w_2classes.csv,SVM,0.715830,0.004014


,file,model_name,acc_test_mean,acc_test_std
0,class_results_wo_2classes.csv,DecisionTree,0.719615,0.041644
1,class_results_wo_2classes.csv,RandomForest,0.732430,0.012212
2,class_results_wo_2classes.csv,SVM,0.707103,0.017171


In [77]:
class_tabla_models_desv(PATH_CLASS,'270_0/run_11_22_1156', 'acc_test')

,file,model_name,acc_test_mean,acc_test_std
0,class_results_w_2classes.csv,DecisionTree,0.754115,0.037100
1,class_results_w_2classes.csv,RandomForest,0.746391,0.019824
2,class_results_w_2classes.csv,SVM,0.754049,0.012694


,file,model_name,acc_test_mean,acc_test_std
0,class_results_wo_2classes.csv,DecisionTree,0.717878,0.045959
1,class_results_wo_2classes.csv,RandomForest,0.733492,0.019260
2,class_results_wo_2classes.csv,SVM,0.729246,0.027643


In [79]:
class_tabla_models_desv(PATH_CLASS,'270_9/run_11_22_1156', 'acc_test')

,file,model_name,acc_test_mean,acc_test_std
0,class_results_w_2classes.csv,DecisionTree,0.770290,0.021887
1,class_results_w_2classes.csv,RandomForest,0.780544,0.024387
2,class_results_w_2classes.csv,SVM,0.769792,0.004944


,file,model_name,acc_test_mean,acc_test_std
0,class_results_wo_2classes.csv,DecisionTree,0.716248,0.057461
1,class_results_wo_2classes.csv,RandomForest,0.741710,0.043108
2,class_results_wo_2classes.csv,SVM,0.724528,0.047985


### Tabla Veces ganador

In [85]:
class_tabla_veces_ganador(PATH_CLASS,'0_0/run_11_22_1156', 'acc_test')

,file,model_name,win_count
0,class_results_w_2classes.csv,DecisionTree,10
1,class_results_w_2classes.csv,RandomForest,4
2,class_results_wo_2classes.csv,RandomForest,14


In [93]:
class_tabla_veces_ganador(PATH_CLASS,'270_0/run_12_10_2117', 'acc_test')

,file,model_name,win_count
0,class_results_w_2classes.csv,SVM,10
1,class_results_w_2classes.csv,DecisionTree,4
2,class_results_wo_2classes.csv,RandomForest,14


In [92]:
class_tabla_veces_ganador(PATH_CLASS,'270_9/run_11_22_1156', 'acc_test')

,file,model_name,win_count
0,class_results_w_2classes.csv,RandomForest,8
1,class_results_w_2classes.csv,SVM,5
2,class_results_w_2classes.csv,DecisionTree,1
3,class_results_wo_2classes.csv,RandomForest,14
